In [2]:
import re

In [3]:
corpus_file = open('Medline/MED.ALL').read()
# split into words
from nltk.tokenize import word_tokenize
tokens = word_tokenize(corpus_file)


In [4]:
# convert to lower case
tokens = [w.lower() for w in tokens]


In [5]:
# remove punctuation from each word
import string

table = string.maketrans(string.punctuation,' '*len(string.punctuation))
stripped = [w.translate(table) for w in tokens]


In [6]:
# remove remaining tokens that are not alphabetic
words = [word for word in stripped if word.isalpha()]


In [7]:
# filter out stop words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
words = [w for w in words if not w in stop_words]
#print(words[:100])
print len(words)
print(len(set(words)))
#print sorted(set(words))

85411
11488


In [8]:
#from nltk.stem.snowball import SnowballStemmer 
#stemmer = SnowballStemmer('english')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
stemmed_words = [ stemmer.stem(word) for word in words]
print len(stemmed_words)
print len(list(set(stemmed_words)))

85411
8033


In [9]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
x = 'added'
stemmer.stem(x)

u'ad'

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
print vectorizer

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


In [11]:
import os
docs = ()
for i in os.listdir('Medline/text_files'):
    i = 'Medline/text_files/'+i
    docs = docs + (open(i).read(),)
docs[0:4]
    

('1384. importance of campimetry and carotid and vertebral angiography in\r\nthrombosis of the posterior cerebral artery\r\nthe authors report a case of thrombosis of the posterior cerebral artery which\r\npresented only lateral homonymous hemianopia and stress the importance of\r\ncampimetry and of carotid angiography beside vertebral angiography.\r',
 'bilateral renal hypoplasia of the pig.\r\nbilateral renal hypoplasia of the pig is described\r\nand is compared with a previous\r\ndescription.  the defect was observed in 19\r\npiglets out of a total of 73 born in 8\r\nlitters, sired by one large white boar.\r\ntwelve affected piglets died at or soon\r\nafter birth and 7 died between the ages of 11\r\nand 69 days.  inheritance by a simple\r\nautosomal recessive factor is suggested by\r\nthe evidence.  grossly the condition\r\nwas characterized by varying degrees of\r\nhypoplasia and persistence of fetal lobulation.\r\nmicroscopically, the dominant feature\r\nwas the poor development o

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
train_set = ("The sky is blue.", "The sun is bright.")
test_set = ("The sun in the sky is bright.",
"We can see the shining sun, the bright sun.")
count_vectorizer = CountVectorizer()
count_vectorizer.fit_transform(train_set)
print "Vocabulary:", count_vectorizer.vocabulary_
# Vocabulary: {'blue': 0, 'sun': 1, 'bright': 2, 'sky': 3}
freq_term_matrix = count_vectorizer.transform(test_set)
print freq_term_matrix.todense()

Vocabulary: {u'blue': 0, u'bright': 1, u'sun': 4, u'is': 2, u'sky': 3, u'the': 5}
[[0 1 1 1 1 2]
 [0 1 0 0 2 2]]


In [13]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(token_pattern = '[a-zA-Z][a-zA-Z]+',min_df=1)
term_freq_matrix = count_vectorizer.fit_transform(docs)
#print "Vocabulary:",count_vectorizer.vocabulary_
voc = count_vectorizer.vocabulary_

#dict(voc.items()[0:2])
#print sorted(voc)
print voc[u'aa']
len(voc)

0


12584

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(norm="l2")
tfidf.fit(term_freq_matrix)

tf_idf_matrix = tfidf.transform(term_freq_matrix)
print tf_idf_matrix.todense()

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(min_df = 1)
tfidf_matrix = tfidf_vectorizer.fit_transform(docs)

print tfidf_matrix.todense()

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [16]:
index_terms = list(set(stemmed_words))
print len(index_terms)

8033


In [17]:
import os
docs = []
for i in os.listdir('Medline/text_files'):
    if i=='doc0':
        continue
    
    i = 'Medline/text_files/'+i
    docs.append(open(i).read())

## index of docs doesnt match with number in doc name
    

In [18]:

doc_index_terms = []
for doc in docs:
    doc_tokens = word_tokenize(doc)
    doc_tokens = [w.lower() for w in doc_tokens]
    doc_stripped = [w.translate(table) for w in doc_tokens]
    doc_words = [word for word in doc_stripped if word.isalpha()]
    doc_words = [w for w in doc_words if w not in stop_words]
    stemmed_words = [ stemmer.stem(word) for word in doc_words]
    doc_index_terms.append(stemmed_words)


In [19]:
doc_index_terms[0]

[u'import',
 u'campimetri',
 u'carotid',
 u'vertebr',
 u'angiographi',
 u'thrombosi',
 u'posterior',
 u'cerebr',
 u'arteri',
 u'author',
 u'report',
 u'case',
 u'thrombosi',
 u'posterior',
 u'cerebr',
 u'arteri',
 u'present',
 u'later',
 u'homonym',
 u'hemianopia',
 u'stress',
 u'import',
 u'campimetri',
 u'carotid',
 u'angiographi',
 u'besid',
 u'vertebr',
 u'angiographi']

In [20]:
def tokenize():
    return

In [21]:
doc_vectors = [[0.0 for j in xrange(len(index_terms))] for i in xrange(len(doc_index_terms))]
for i in xrange(len(doc_index_terms)):
    for j in xrange(len(index_terms)):
        doc_vectors[i][j] = doc_index_terms[i].count(index_terms[j])
        



In [22]:
import nltk
import string
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer

path = 'Medline/text_files/'
token_dict = {}
stemmer = PorterStemmer()

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token for token in tokens if token.isalpha()]
    stems = stem_tokens(tokens, stemmer)
    return stems

for subdir, dirs, files in os.walk(path):
    for file in files:
        file_path = subdir + os.path.sep + file
        shakes = open(file_path, 'r')
        text = shakes.read()
        lowers = text.lower()
        no_punctuation = lowers.translate(None, string.punctuation)
        token_dict[file] = no_punctuation
        
#this can take some time
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
tfs = tfidf.fit_transform(token_dict.values())




In [38]:
counter = CountVectorizer(tokenizer=tokenize, stop_words='english')
counter_ = counter.fit_transform(token_dict.values())

In [40]:
counter_.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [55]:
counter_ = counter_.toarray()

In [23]:
print token_dict.items()[0:3]
print token_dict.values()[0:3]

[('doc365', '2469 the nature of collagen disease particularly of systemic lupus\r\nerythematosus sle with special reference to renal lesions japanese\r\nfrom the results of examination of 161 autopsy cases of collagen diseases\r\nrheumatic fever and polyarteritis nodosa pn are considered as diseases of\r\nhyperergic nature showing pronounced specific manifestation in particular\r\norgans  as the reactivity becomes lower this specific localization becomes\r\nless significant and many organs become affected in sle  it may accordingly be\r\nsupposed that sle is not a disease of hypersensitivity but one occurring in\r\na state of exhaustion of reactability after prolonged sensitization  moreover\r\na slelike syndrome arises not only in the end stage of parasepsis nephritis\r\nnephrosis and some cases of prolonged sensitization by mycotb but also in\r\npnpssdm and rheumatism  it may therefore be conceivable that sle is\r\na syndrome rather than a separate entity  it is most important to con

In [24]:
print tfs

  (0, 6721)	0.0675480662571
  (0, 3700)	0.114130814134
  (0, 4688)	0.0977733779356
  (0, 6296)	0.0816785794425
  (0, 2574)	0.10561169878
  (0, 5343)	0.0693430389469
  (0, 4490)	0.102571094899
  (0, 714)	0.191618185256
  (0, 2711)	0.0824922603101
  (0, 6726)	0.0618397079806
  (0, 7136)	0.0675480662571
  (0, 8258)	0.0924624774165
  (0, 4040)	0.0595826000732
  (0, 2667)	0.0940519795449
  (0, 7594)	0.0672111426845
  (0, 1616)	0.10561169878
  (0, 8441)	0.0762956392883
  (0, 6466)	0.120892816405
  (0, 5425)	0.120892816405
  (0, 5532)	0.114130814134
  (0, 5527)	0.10561169878
  (0, 6082)	0.120892816405
  (0, 7942)	0.0653211432441
  (0, 2624)	0.0824922603101
  (0, 5859)	0.0461291121246
  :	:
  (1033, 7072)	0.076489384575
  (1033, 6208)	0.0628042664979
  (1033, 6728)	0.0530690121087
  (1033, 6355)	0.0713024073155
  (1033, 5032)	0.0529226790665
  (1033, 373)	0.0688284077588
  (1033, 3139)	0.0450764082165
  (1033, 789)	0.0704357672832
  (1033, 2254)	0.15297876915
  (1033, 2245)	0.0620263725503
  (

In [25]:
token_dict.items()[0:10]

[('doc365',
  '2469 the nature of collagen disease particularly of systemic lupus\r\nerythematosus sle with special reference to renal lesions japanese\r\nfrom the results of examination of 161 autopsy cases of collagen diseases\r\nrheumatic fever and polyarteritis nodosa pn are considered as diseases of\r\nhyperergic nature showing pronounced specific manifestation in particular\r\norgans  as the reactivity becomes lower this specific localization becomes\r\nless significant and many organs become affected in sle  it may accordingly be\r\nsupposed that sle is not a disease of hypersensitivity but one occurring in\r\na state of exhaustion of reactability after prolonged sensitization  moreover\r\na slelike syndrome arises not only in the end stage of parasepsis nephritis\r\nnephrosis and some cases of prolonged sensitization by mycotb but also in\r\npnpssdm and rheumatism  it may therefore be conceivable that sle is\r\na syndrome rather than a separate entity  it is most important to c

In [26]:
tfidf_matrix = tfs.todense()

In [27]:
tfidf.vocabulary_

{u'degenera': 2039,
 u'glisson': 3365,
 u'yellow': 9442,
 u'immunohematolo': 4018,
 u'communica': 1572,
 u'perineur': 6228,
 u'amniot': 337,
 u'accret': 56,
 u'malefemal': 4930,
 u'albumen': 256,
 u'serolo': 7616,
 u'kwashiorkor': 4532,
 u'roent': 7360,
 u'maladapt': 4920,
 u'reemphas': 7124,
 u'potassiqu': 6570,
 u'pigment': 6367,
 u'lumin': 4829,
 u'leukon': 4663,
 u'polypeptid': 6505,
 u'cardiomegali': 1138,
 u'disturb': 2316,
 u'wooden': 9392,
 u'liation': 4680,
 u'familycontact': 2937,
 u'pitressin': 6384,
 u'fect': 2969,
 u'pubic': 6880,
 u'second': 7519,
 u'bili': 867,
 u'accumu': 59,
 u'cholangiolit': 1354,
 u'dialogu': 2178,
 u'vacuol': 9070,
 u'specialist': 7858,
 u'intellectu': 4229,
 u'selec': 7545,
 u'ophthalm': 5880,
 u'error': 2730,
 u'lancefield': 4568,
 u'pancreat': 6040,
 u'mediastinum': 5045,
 u'herp': 3677,
 u'iq': 4359,
 u'theless': 8428,
 u'duoden': 2426,
 u'deterior': 2140,
 u'postop': 6553,
 u'k': 4456,
 u'virusin': 9229,
 u'immunoselect': 4024,
 u'multivesicula

In [56]:
print tfidf_matrix.shape
tfidf_matrix
len(tfidf_matrix)


(1034, 9468)


1034

In [57]:
doc_transactions = [[] for x in xrange(len(counter_))]
for i in xrange(counter_.shape[0]):
    for j in xrange(counter_.shape[1]):
        if counter_[i][j]:
            doc_transactions[i].append(j)
            
print doc_transactions



[[54, 195, 568, 714, 724, 806, 1154, 1533, 1616, 1667, 2281, 2574, 2624, 2667, 2711, 2734, 2819, 2842, 3001, 3700, 3858, 3879, 4040, 4416, 4490, 4645, 4688, 4774, 4814, 4832, 4960, 4962, 5339, 5343, 5425, 5471, 5527, 5532, 5618, 5804, 5859, 5899, 6082, 6116, 6118, 6296, 6453, 6466, 6483, 6721, 6726, 6757, 6768, 7054, 7057, 7127, 7136, 7184, 7252, 7294, 7585, 7594, 7706, 7775, 7777, 7857, 7861, 7942, 7964, 8193, 8258, 8262, 8441, 8449], [395, 737, 868, 893, 902, 1186, 1598, 1777, 1828, 1829, 1854, 2040, 2463, 2844, 3556, 3644, 3764, 4419, 4759, 4813, 4929, 5789, 5990, 6719, 6825, 7612, 7884, 8262, 8826, 9419], [85, 109, 530, 560, 562, 667, 704, 775, 792, 865, 868, 902, 924, 1154, 1209, 1215, 1278, 1424, 1534, 1647, 1665, 1667, 1733, 1942, 1946, 1969, 2265, 2299, 2316, 2420, 2432, 2656, 2682, 2737, 2789, 2894, 2950, 2966, 2997, 3027, 3028, 3072, 3087, 3196, 3338, 3448, 3469, 3483, 3663, 3726, 3950, 4077, 4088, 4128, 4133, 4140, 4253, 4302, 4318, 4419, 4445, 4654, 4690, 4729, 4759, 4853, 

In [61]:
save_transactions = open('transactions','w')
import json
doc_transactions_json = {"transactions":doc_transactions}
json.dump(doc_transactions, save_transactions)

In [ ]:
from sklearn.cluster import KMeans
#import numpy as np

kmeans = KMeans(n_clusters=10, random_state=0).fit(tfidf_matrix)
print kmeans.labels_


print kmeans.cluster_centers_
